In [4]:

%run -i ./startup.py

%run -i ./functions/dc_transform.py

%run -i ./functions/dc_join.py

data_collections = ['^cursos*','estruturas_curriculares','curriculo_componente','componentes_curriculares','turmas','matriculas_em_componentes']

for dc in data_collections:
    print(dc)
    dc_join(db=db,dc=dc)


^cursos*
estruturas_curriculares
curriculo_componente
componentes_curriculares
turmas
matriculas_em_componentes


In [5]:
import re
for dcb in dc_join.dict.keys():
    print('%-35s' % dcb,end='') # %-35s : varchar 35 alinhado à esquerda
    c = []
    for k in dc_join.dict[dcb]['data'].keys():
        if re.findall('^id_.*',k):
            dc_join.dict[dcb]['data'][k] = dc_join.dict[dcb]['data'][k].astype('Int64')
            c.append(k)

    print(c,end='')
    print('')

ufrn_wait/cursos                   ['id_curso', 'id_coordenador', 'id_unidade_responsavel']
ufrn_wait/estruturas_curriculares  ['id_curriculo', 'id_curso']
ufrn_wait/curriculo_componente     ['id_curriculo_componente', 'id_curriculo', 'id_componente_curricular']
ufrn_wait/componentes_curriculares ['id_componente']
ufrn_wait/turmas                   ['id_turma', 'id_componente_curricular', 'id_turma_agrupadora']
ufrn_wait/matriculas_em_componentes['id_turma', 'id_curso']


In [6]:
dc = dc_join.dict

cursos                   = dc['ufrn_wait/cursos']['data']
estruturas_curriculares  = dc['ufrn_wait/estruturas_curriculares']['data']
curriculo_componente     = dc['ufrn_wait/curriculo_componente']['data']
componentes_curriculares = dc['ufrn_wait/componentes_curriculares']['data']
turmas                   = dc['ufrn_wait/turmas']['data']
matriculas_componentes   = dc['ufrn_wait/matriculas_em_componentes']['data']

### Análise de notas por estrutura curricular

In [8]:
import numpy as np

,index,id_curso,nome,id_coordenador,coordenador,situacao_curso,nivel_ensino,grau_academico,modalidade_educacao,area_conhecimento,...,unidade_responsavel,website,data_funcionamento,codigo_inep,dou,portaria_reconhecimento,convenio_academico,programa,portaria,conceito
0,1140,92127264,TECNOLOGIA DA INFORMAÇÃO,5756813,RAFAELA HORACINA SILVA ROCHA SOARES,ATIVO,GRADUAÇÃO,BACHARELADO,Presencial,Ciências Exatas e da Terra,...,INSTITUTO METROPOLE DIGITAL,NaN,NaN,1190414.0,2016/05/02 00:00:00.000,Portaria nº 127/2016 - MEC,NaN,NaN,NaN,NaN
1,1273,92127264,TECNOLOGIA DA INFORMAÇÃO,5757620,ISABEL DILLMANN NUNES,ATIVO,GRADUAÇÃO,BACHARELADO,Presencial,Ciências Exatas e da Terra,...,INSTITUTO METROPOLE DIGITAL,NaN,NaN,1190414.0,2016/05/02 00:00:00.000,Portaria nº 127/2016 - MEC,NaN,NaN,NaN,NaN


In [87]:
getin = lambda df,i,v: df[df[i].isin(v.unique())].reset_index(drop=True)
getre = lambda df,i,v: df[df[i].isin(list(filter(lambda i : re.findall(v,i),df[i])))].reset_index(drop=True)

#qri_curso        = getre(cursos,'nome','^CIÊNCIA.*LOGIA')
qri_curso        = getre(cursos,'nome','^TEC.*INFORMAÇÃO$')

qri_curso_ec     = getin(estruturas_curriculares, 'id_curso',qri_curso.id_curso)
qri_curso_cc     = getin(curriculo_componente,'id_curriculo',qri_curso_ec.id_curriculo)
qri_curso_ccc    = getin(componentes_curriculares,'id_componente',qri_curso_cc.id_componente_curricular)
qri_curso_turmas = getin(turmas,'id_componente_curricular',qri_curso_ccc.id_componente)
qri_curso_mc     = getin(matriculas_componentes,'id_turma',qri_curso_turmas.id_turma)

qri_curso_turmas_unique = qri_curso_turmas[~qri_curso_turmas.id_turma.duplicated()]

A = qri_curso_turmas_unique[['id_turma','id_componente_curricular','ano','periodo','situacao_turma']]
B = qri_curso_mc
C = pd.merge(A,B,on=['id_turma'])
D = qri_curso_ccc[['id_componente','codigo','modalidade','nome']]
D = D.rename(columns={'id_componente':'id_componente_curricular'})
D = pd.merge(C,D,on=['id_componente_curricular'])
D = D[D['situacao_turma']=='CONSOLIDADA']

nota2float = lambda nota : float(nota.replace(',','.')) if isinstance(nota,str) else nota

D.nota                = D.nota.transform(nota2float)
D.media_final         = D.media_final.transform(nota2float)
D.unidade             = D.unidade.astype('Int64')
D.faltas_unidade      = D.faltas_unidade.astype('Int64')
D.numero_total_faltas = D.numero_total_faltas.astype('Int64')

E = D[['id_turma','id_componente_curricular','descricao','ano','periodo','unidade','nota','faltas_unidade','nome','id_curso','discente']]
E = E.sort_values(by=['ano','periodo','nome'])

# u_ unique values
# n_ number of values
# i_ index dict

n_i = E.id_componente_curricular.nunique()
#n_a = E.ano.nunique()
#n_p = E.periodo.nunique()
n_ap = (E.ano*10 + E.periodo).unique().size
n_u = E.unidade.nunique()

i_e = {k:v for v,k in enumerate(E.id_componente_curricular.unique())}

c = 0

notas_componentes    = np.empty((n_i, n_ap, n_u, 8))
notas_componentes[:] = np.nan
description_keys = ['count', 'mean', 'std', 'min', '25%', '50%', '75%', 'max']

c = 0
apk = 0
ap_dict = {}
for a,ano in enumerate(E.ano.unique()):
    _a = E[E.ano==ano]
    print(ano,end=' ')
    for p,periodo in enumerate(_a.periodo.unique()):
        _ap = _a[_a.periodo==periodo];
        u_i = _ap.id_turma.unique()
        
        ap_key = ano*10+periodo
        if not ap_key in ap_dict:
            ap_dict[ap_key] = apk
            apk = apk + 1;
        
        ap = ap_dict[ap_key] 
            
        for t,turma_id in enumerate(u_i):
            _apt = _ap[_ap.id_turma == turma_id];
            _apt = _apt[~(_apt.descricao == 'INDEFERIDO')]
            _apt = _apt[~pd.isna(_apt.unidade)]
            
            for u,unidade in enumerate(_apt.unidade.unique()):
                _aptu = _apt[(_apt.unidade == unidade)]
                i = i_e[_aptu.id_componente_curricular.iloc[0]]
                
                if _aptu.nota.size:
                    n_d = _aptu.size
                    notas_componentes[i][ap][u] = _aptu.nota.describe()
                    
                    c = c + 1

pa_dict = {ap_dict[k]:k for k in ap_dict.keys()}

2009 2010 2011 2012 2013 2014 2015 2016 2017 2018 2019 2020 

In [88]:
from matplotlib import pyplot as plt
import seaborn as sns

In [89]:
kap = lambda k : str(k//10) + '.' + str(k - (k//10)*10)

periodo_labels = [kap(k) for k in ap_dict.keys()]
periodo_n      = len(periodo_labels)
periodo_ticks  = np.arange(periodo_n)

In [93]:
nota_bin_diff    = .1
nota_bin_edges   = np.arange(-nota_bin_diff,10+nota_bin_diff,nota_bin_diff)
nota_bin_edges_n = nota_bin_edges.size

nota_bin_center = [np.mean(nota_bin_edges[i:i+2]) for i in range(nota_bin_edges_n-1)]

nota_bin_ticks_n         = nota_bin_edges_n-1
nota_bin_ticks           = np.arange(nota_bin_ticks_n)
nota_bin_tlabels_inteval = ['(%.1f, %2.1f]' % tuple(nota_bin_edges[i:i+2]) for i in range(nota_bin_ticks_n)]
nota_bin_tlabels_center  = ['%.1f' % np.mean(nota_bin_edges[i:i+2]) for i in range(nota_bin_ticks_n)]
nota_bin_tlabels_closed  = ['%.1f' % nota_bin_edges[i+1] for i in range(nota_bin_ticks_n)]

In [149]:
np.nanmean(np.nanmean(notas_componentes[:,:,:,1],axis=2),axis=1)

<ipython-input-149-18f0e1b97a0e>:1: RuntimeWarning: Mean of empty slice
  np.nanmean(np.nanmean(notas_componentes[:,:,:,1],axis=2),axis=1)


array([ 8.48444444,  5.63871045,  5.86361243,  4.64622569,  6.63787181,
        5.84273044,  7.09047619,  6.0673517 ,  4.80544953,  4.38053412,
        3.79338449,  5.20026742,  6.26248419,  6.12725847,  7.14348148,
        7.01388889,  6.06139704,  6.54983884,  6.75346093,  5.58888889,
        7.92837302,  6.88250096,  6.05079365,  5.51536267,  8.19886775,
        7.23333333,  6.34814815,  7.11817296,  5.68686483,  7.38531205,
        6.84442708,  6.77809491,  5.66909839,  5.17032554,  4.75344886,
        5.9934228 ,  7.89063959,  7.3445709 ,  5.37765131,  6.72443899,
        7.28689735,  6.59556552,  5.45530864,  6.7917118 ,  6.43955127,
        5.64550095,  7.36707358,  6.76524735,  5.99409188,  6.0792438 ,
        8.01597831,  6.90496837,  7.05395589,  3.61228725,  6.23509804,
        6.68262216,  5.65449072,  3.0943484 ,  2.5237458 ,  5.63191001,
        5.34906163,  6.05710492,  5.91736172,  6.49375858,  5.68461959,
        7.53821466,  6.33794753,  3.55943968,  5.61740555,  7.01

In [146]:
data = np.histogram(np.nanmean(np.nanmean(notas_componentes[:,:,:,1],axis=2),axis=1),nota_bin_edges)

%matplotlib widget

plt.close(1)
sns.set_theme(context='notebook',palette='colorblind',style='ticks')

fig,ax = plt.subplots(1,1,figsize=(15.5,5),num=1)
h_bar = sns.barplot(x=data[1][1:], y=data[0],ax=ax,facecolor=(1,1,1),edgecolor=(0,0,0))

for n,nlbl in enumerate(nota_bin_tlabels_closed):
    if len(nlbl) and nlbl[-1]!='0':
        nota_bin_tlabels_closed[n] = ''
xtl = ax.set_xticklabels(nota_bin_tlabels_closed)

ax.set_ylabel('Número de componentes')
ax.set_xlabel('Média das notas')
ax.set_title('Nota média dos componentes do BTI')
ax.set_position((.1,.3,.8,.6))

ax.set_ylim((0,15))

<ipython-input-146-540865896497>:1: RuntimeWarning: Mean of empty slice
  data = np.histogram(np.nanmean(np.nanmean(notas_componentes[:,:,:,1],axis=2),axis=1),nota_bin_edges)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(0.0, 15.0)

In [150]:
ax.set

[Text(0, 0, '0.0'),
 Text(1, 0, ''),
 Text(2, 0, ''),
 Text(3, 0, ''),
 Text(4, 0, ''),
 Text(5, 0, ''),
 Text(6, 0, ''),
 Text(7, 0, ''),
 Text(8, 0, ''),
 Text(9, 0, ''),
 Text(10, 0, '1.0'),
 Text(11, 0, ''),
 Text(12, 0, ''),
 Text(13, 0, ''),
 Text(14, 0, ''),
 Text(15, 0, ''),
 Text(16, 0, ''),
 Text(17, 0, ''),
 Text(18, 0, ''),
 Text(19, 0, ''),
 Text(20, 0, '2.0'),
 Text(21, 0, ''),
 Text(22, 0, ''),
 Text(23, 0, ''),
 Text(24, 0, ''),
 Text(25, 0, ''),
 Text(26, 0, ''),
 Text(27, 0, ''),
 Text(28, 0, ''),
 Text(29, 0, ''),
 Text(30, 0, '3.0'),
 Text(31, 0, ''),
 Text(32, 0, ''),
 Text(33, 0, ''),
 Text(34, 0, ''),
 Text(35, 0, ''),
 Text(36, 0, ''),
 Text(37, 0, ''),
 Text(38, 0, ''),
 Text(39, 0, ''),
 Text(40, 0, '4.0'),
 Text(41, 0, ''),
 Text(42, 0, ''),
 Text(43, 0, ''),
 Text(44, 0, ''),
 Text(45, 0, ''),
 Text(46, 0, ''),
 Text(47, 0, ''),
 Text(48, 0, ''),
 Text(49, 0, ''),
 Text(50, 0, '5.0'),
 Text(51, 0, ''),
 Text(52, 0, ''),
 Text(53, 0, ''),
 Text(54, 0, ''),
 T

In [13]:
data         = np.nanmean(np.nanmean(notas_componentes[:,:,:,1],axis=2),axis=1)
data_count,_ = np.histogram(data,bins=nota_bin_edges)

<ipython-input-13-8e184ef556dc>:1: RuntimeWarning: Mean of empty slice
  data         = np.nanmean(np.nanmean(notas_componentes[:,:,:,1],axis=2),axis=1)


In [14]:
%matplotlib widget

plt.close(1)
sns.set_theme(context='notebook',palette='colorblind',style='ticks')

fig,ax = plt.subplots(1,1,figsize=(7.5,3),num=1)

h_bar = sns.barplot(x=nota_bin_edges[1:], y=data_count,ax=ax,facecolor=(1,1,1),edgecolor=(0,0,0))

xtl = ax.set_xticklabels(nota_bin_tlabels_closed)
ax.set_ylabel('Número de componentes')
ax.set_xlabel('Média das notas')
ax.set_title('Desempenho médio dos discentes nos componentes da ECT')
ax.set_position((.1,.3,.8,.6))
ax.set_ylim((0,50))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(0.0, 50.0)

In [27]:
notas_cap = np.nanmean(notas_componentes[:,:,:,1],axis=2)

<ipython-input-27-d516d6d32e4b>:1: RuntimeWarning: Mean of empty slice
  notas_cap = np.nanmean(notas_componentes[:,:,:,1],axis=2)


(279, 23)

In [49]:
notas_cap_count = np.empty((n_i, nota_bin_ticks_n))

for c in range(n_i):
    notas_cap_ = notas_cap[c,~pd.isna(notas_cap[c,:])]

    notas_cap_count[c,:],_ = np.histogram(notas_cap_, bins=nota_bin_edges)

In [50]:
%matplotlib widget

import matplotlib.colors as colors

plt.close(2)
sns.set_theme(context='notebook',palette='colorblind',style='ticks')


#data_count / np.sum(data_count,axis=0)
#np.random.random( ( 5, 5 ) ) / np.arange( 5 )

fig = plt.figure(num=2,figsize=(8,6))
ax = plt.subplot()
cm_norm = colors.BoundaryNorm(boundaries=np.arange(0,.31,.01),ncolors=300,clip=True)

plt.imshow(notas_cap_count,norm=cm_norm,cmap='viridis')
#plt.imshow(data_count/np.sum(data_count,axis=0),cmap='viridis')

ax.set_ylabel('Intervalo das notas')
ax.set_xlabel('Perídos')
ax.set_yticks(np.arange(-nota_bin_diff,21.,nota_bin_diff*2))
ax.set_xticks(periodo_ticks)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [47]:
%matplotlib widget

import matplotlib.colors as colors

plt.close(2)
sns.set_theme(context='notebook',palette='colorblind',style='ticks')


#data_count / np.sum(data_count,axis=0)
#np.random.random( ( 5, 5 ) ) / np.arange( 5 )

fig = plt.figure(num=2,figsize=(8,6))
ax = plt.subplot()
cm_norm = colors.BoundaryNorm(boundaries=np.arange(0,.31,.01),ncolors=300,clip=True)

plt.imshow(im_matrix,norm=cm_norm,cmap='viridis')
#plt.imshow(data_count/np.sum(data_count,axis=0),cmap='viridis')

ax.set_ylabel('Intervalo das notas')
ax.set_xlabel('Perídos')
ax.set_yticks(range(nota_bin_ticks_n))
ax.set_xticks(periodo_ticks)
ytl = ax.set_xticklabels(periodo_labels)
xtl = ax.set_yticklabels(nota_bin_tlabels_inteval)

[y.set_rotation(40) for y in ytl];

ax.set_title('Distribuição de probabilidade das notas ao longo dos perídos')
cbar = plt.colorbar(ticks=np.arange(0,.31,.05))
cbar.solids.set_edgecolor("face")
ax.set_position((-0.2,.2,.95,.7))

cbar.set_label('Probabilidade')


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [53]:
%matplotlib widget

import matplotlib.colors as colors

plt.close(2)
sns.set_theme(context='notebook',palette='colorblind',style='ticks')

data = np.nanmean(notas_componentes[:,:,:,1],axis=2)

data_count = np.empty((nota_bin_ticks_n,n_ap))

for ap in range(n_ap):
    i_d = ~pd.isna(data[:,ap])
    (data_count[:,ap],_) = np.histogram(data[i_d,ap],bins=nota_bin_edges)


#data_count / np.sum(data_count,axis=0)
#np.random.random( ( 5, 5 ) ) / np.arange( 5 )

fig = plt.figure(num=2,figsize=(8,6))
ax = plt.subplot()

cm_max = 40
cm_norm = colors.BoundaryNorm(boundaries=np.arange(0,cm_max,1),ncolors=300,clip=True)

plt.imshow(data_count,norm=cm_norm,cmap='viridis')
#plt.imshow(data_count/np.sum(data_count,axis=0),cmap='viridis')

ax.set_ylabel('Intervalo das notas')
ax.set_xlabel('Perídos')
ax.set_yticks(range(nota_bin_ticks_n))
ax.set_xticks(periodo_ticks)
ytl = ax.set_xticklabels(periodo_labels)
xtl = ax.set_yticklabels(nota_bin_tlabels_inteval)

[y.set_rotation(40) for y in ytl];

ax.set_title('Descrição ao longo dos períodos da\ncontagem de componentes curriculares\n que a média das notas dos estudantes foi igual a do intervalo')
cbar = plt.colorbar(ticks=np.arange(0,40,5))
cbar.solids.set_edgecolor("face")
ax.set_position((-0.2,.2,.95,.7))

cbar.set_label('Número de componentes curriculares')

<ipython-input-53-a66d29fd5ab9>:8: RuntimeWarning: Mean of empty slice
  data = np.nanmean(notas_componentes[:,:,:,1],axis=2)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
notas_ca = np.mean(notas_componentes[:,:,:,1],axis=2)

In [ ]:
%notebook widget

from matplotlib import pyplot as plt
fnum = 2
plt.close(fnum)

plt.figure(num=fnum,figsize=(16,5))

a = plt.subplot()

yquantiles = np.nanquantile( notas_ca, [.25, .5, .75], axis=0 ).transpose()
ymean      = np.nanmean( notas_ca, axis=0 )

plt.plot( yquantiles ,'ok-')
plt.plot( ymean ,'o-')

a.set_ylim([0,10])
a.set_xticks(np.arange(0,n_ap))
a.set_xticklabels(tap)
a.set_position((.1,.1,.8,.8))

a.set_ylabel('Notas')
a.set_xlabel('Ano e período')
a.set_title('Média da média da média das notas dos discentes nas unidades dos componentes através dos períodos letivos')

In [ ]:
notas_ca[:,:12].shape
notas_ca[:,:12].shape

In [ ]:
notas_antes = np.nanmean(notas_ca[:,:12])
notas_depois = np.nanmean(notas_ca[:,12:])

In [ ]:
(1- notas_depois/notas_antes)*100

In [379]:
#z = sns.histplot(data,bins=data_bin)

z.set_position((0.1,0.3,.8,.5))
z.set_ylabel('Nº de componentes')
z.set_xlabel('Nota média')
z.set_title('A média no tempo da média das unidades da média dos discentes em cada componente do curso da ECT')

ax.text(5,5,'ok')
class Cursor:
    def __init__(self, ax):
        self.ax = ax
        self.lx = ax.axhline(color='k')  # the horiz line
        self.ly = ax.axvline(color='k')  # the vert line
                  
        # text location in axes coords
        self.txt = ax.text(.1, .8, '', transform=ax.transAxes)

    def mouse_move(self, event):
        if not event.inaxes:
            return

        x, y = event.xdata, event.ydata
        # update the line positions
        self.lx.set_ydata(y)
        self.ly.set_xdata(x)

        self.txt.set_text('x=%1.2f, y=%1.2f' % (x, y))
        self.ax.figure.canvas.draw()

h_event = Cursor(ax)
fig.canvas.mpl_connect('motion_notify_event',h_event.mouse_move)

AttributeError: 'list' object has no attribute 'set_position'

In [410]:
rr = [ ]

n_periodos = np.empty((n_i,))

for n in range(n_i):

    notas    = notas_componentes[n,:,:,1]
    
    id_pa    = ~np.all( np.isnan(notas),axis=1)
    id_notas = np.where(id_pa)[0]
    
    notas    = notas[id_notas,:]
    ano_periodo = np.array([pa_dict[k] for k in id_notas])
    
    n_periodos[n] = len(ano_periodo)
    
    r = np.corrcoef(ano_periodo,np.nanmean(notas,axis=1))[0,1]
   
    rr.append(r)

In [411]:
ap_mean = np.empty(n_a*n_p)
for ap,apm in enumerate(ap_mean):
    ap_mean[ap] = np.nanmean(np.nanmean(notas_componentes[:,ap,:,1],axis=1))

NameError: name 'n_a' is not defined

In [364]:
%matplotlib widget

from matplotlib import pyplot as plt
import seaborn as sns
plt.close(1)

f,a = plt.subplots(1,1,figsize=(7,2),num=1)
z = sns.histplot(n_periodos,bins=np.arange(0,25,1),ax=a)
z.set_position((0.1,0.3,.8,.5))
z.set_ylabel('Nº de ofertas')
z.set_xlabel('Nº de semestres')
z.set_title('Quantidade de oferta de componentes por semestre')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

NameError: name 'n_periodos' is not defined

In [363]:
irn = sorted( enumerate( zip( rr, n_periodos ) ), key = lambda r: r[1][0] )

irn = np.array([[x[0], *x[1]] for x in irn])

irn = irn[irn[:,2]>5,:]

NameError: name 'rr' is not defined

In [362]:
%matplotlib widget

from matplotlib import pyplot as plt
import seaborn as sns

f,a = plt.subplots(1,1,figsize=(7,2))
z = sns.histplot(n_periodos,bins=np.arange(0,25,1),ax=a)
z.set_position((0.1,0.3,.8,.5))
z.set_ylabel('Nº de ofertas')
z.set_xlabel('Nº de semestres')
z.set_title('Quantidade de oferta de componentes por semestre')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

NameError: name 'n_periodos' is not defined

In [361]:
%matplotlib widget

f,a = plt.subplots(1,1,figsize=(5,5))

nn = 1
#for nn in range(20):#nn = 3
n = int(irn[nn,0])

#   0        1        2        3        4        5        6        7     
# count     mean     std      min      25%      50%      75%      max    

notas1 = notas_componentes[n,:,:,1]
notas_id_nan = ~np.all(np.isnan(notas1),axis=1)
id_notas = np.where(notas_id_nan)[0]
notas = notas1[id_notas,:]
ano_periodo = np.array([pa_dict[k] for k in id_notas])
a.plot(ano_periodo, np.nanmean(notas,axis=1),'o-')
a.set_ylim((0,10))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

NameError: name 'irn' is not defined

In [360]:
from matplotlib import pyplot as plt
import seaborn as sns
f,a = plt.subplots(1,1,figsize=(5,5))

for nn in range(10):#nn = 3
    nnn = 278-nn
    u_i_e = E.id_componente_curricular.unique()
    i_r_m = [int(id_r[:,0][nnn])]
    u_i_m = u_i_e[i_r_m][0]

    n = i_e[u_i_m]
    notas1 = iapu_notas_description[n,:,:,5]
    notas_id_nan = ~np.all(np.isnan(notas1),axis=1)
    id_notas = np.where(notas_id_nan)[0]
    notas = notas1[id_notas,:]
    ano_periodo = np.array([pa_dict[k] for k in id_notas])
    a.plot(ano_periodo, np.nanmean(notas,axis=1),'o-')


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

NameError: name 'id_r' is not defined

36

In [669]:
rr = np.array(rr)

In [ ]:
iapu_notas_description

In [ ]:
np.sort(iapu_notas_description

In [689]:
%matplotlib widget
f,a = plt.subplots(1,1,figsize=(7,7))
a.plot(rr)
a.plot([0,rr.size],[-1,1])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [674]:
plt.plot(range(len(rr)),rr)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [672]:
%matplotlib widget
plt.plot(rr)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [359]:
pa_dict = {ap_dict[k]:k for k in ap_dict.keys()}

notas = iapu_notas_description[4,:,:,1]
notas_id_nan = ~np.all(np.isnan(notas),axis=1)
id_notas = np.where(notas_id_nan)[0]
notas = notas[id_notas,:]
ano_periodo = [pa_dict[k] for k in id_notas]

r = np.corrcoef(ano_periodo,np.mean(notas,axis=1))[0,1]

%matplotlib widget

import seaborn as sns
from matplotlib import pyplot as plt

f,a = plt.subplots(1,1,figsize=(8,3))
#a.plot(ano_periodo,notas[:,0])
a.plot(ano_periodo,np.mean(notas,axis=1))
np.mean(notas,axis=1)

a.set_ylim([0, 10])


NameError: name 'iapu_notas_description' is not defined

In [357]:
z = []
x = []
x.append('')
s = _aptu.nome.iloc[0]
S = s.split()

In [354]:
# Each line with 50 characteres
# or more up to 60 characteres not to split a string

s = _aptu.nome.iloc[0]

r = re.findall('(\w+\s?\w+\s?\w+\s?\w+\s?\w+\s?\w+\s?\w+)\s?',s)

print( len(s), s, sep='\n' )

for r in r:
    print(r)


23
TÉCNICAS DE MICROSCOPIA
TÉCNICAS DE MICROSCOPIA
